In [1]:
5+5

10

In [4]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\akhil.goud.bathini\.conda\envs\chro\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [6]:
extracted_data=load_pdf("data/")

In [7]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)

In [9]:
def download_embedding_model():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embedding=download_embedding_model()

c:\Users\akhil.goud.bathini\.conda\envs\chro\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\akhil.goud.bathini\.conda\envs\chro\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [12]:
from langchain import embeddings
persist_directory='db'
vectordb=Chroma.from_documents(documents=text_chunks[:1000],embedding=embedding,persist_directory=persist_directory)

In [13]:
persist_directory='db'

In [14]:
vectordb=None

In [15]:
vectordb=Chroma(persist_directory=persist_directory,embedding_function=embedding)

In [16]:
retriever=vectordb.as_retriever()

In [19]:
docs=retriever.get_relevant_documents("what is Endoscopy")

In [20]:
docs

[Document(page_content='light source is passed into the esophagus. Endoscopy isused to look directly at the surface of the esophagus.This test can also detect tumors that cause symptomslike those of achalasia. Cancer of the esophagus occurs\nas a complication of achalasia in 2-7% of patients.\nTreatment\nThe first-line treatment for achalasia is balloon dila-', metadata={'page': 33, 'source': 'data\\Medical_book.pdf'}),
 Document(page_content='rial toxins or poisons produced by different strainsof the bacterium Clostridium botulinum . The tox-\nins cause muscle paralysis.\nDysphagia —Difficulty in swallowing.\nEndoscopy —A test in which a viewing device and\na light source are introduced into the esophagus bymeans of a flexible tube. Endoscopy permits visualinspection of the esophagus for abnormalities.\nEsophageal manometry —A test in which a thin', metadata={'page': 34, 'source': 'data\\Medical_book.pdf'}),
 Document(page_content='• Esophageal manometry. In this test, a thin tube is\

In [21]:
llm=CTransformers(model="Model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [22]:
prompt_template="""
    use the following piece of information to answer the user's question.
    if you are not aware of it, just say you are not aware of it, don't try to make up an answer.
    Context:{context}
    Question:{question}
    
    only return the helpful answer.
    Helpful answer:"""

In [23]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [24]:
from langchain.chains import RetrievalQA

In [25]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [26]:
while True:
    user_input=input(f"Input prompt:")
    result=qa({"query":user_input})
    print("Response : ",result["result"])

c:\Users\akhil.goud.bathini\.conda\envs\chro\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response :  
    The answer is on page 83 of the document provided, in the section titled "New York".
Response :  
The umbilical cord connects the developing fetus tothe placenta, which provides oxygen and nutrients.During pregnancy, the umbilical cord contains a rich supplyof blood vessels that nourish the growing fetus. Afterbirth, the umbilical cord is cut, leaving a small stumpattached to the baby's navel. The stump eventually dries outand falls off, usually within two weeks of birth.
    
    Please let me know if you have any other questions.
